In [1]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
import json
from pandas.io.json import json_normalize
from mlxtend.frequent_patterns import association_rules

In [2]:
data = json.load(open('2017-12.json'))

In [3]:
rawDf = json_normalize(data['games'])
rawDf.head()

,added,card_history,coin,duration,hero,hero_deck,id,legend,mode,note,opponent,opponent_deck,rank,region,result,user_hash
0,2017-12-01T06:33:28Z,"[{'player': 'me', 'turn': 1, 'card': {'id': 'U...",False,521.0,Mage,Burn,97911097,None,casual,None,Paladin,Midrange,NaN,Americas,win,66FFB43E8DB8910A32FB03BD640103F6
1,2017-12-01T06:24:33Z,"[{'player': 'opponent', 'turn': 1, 'card': {'i...",False,666.0,Priest,None,97910783,None,casual,None,Shaman,None,NaN,Americas,loss,66FFB43E8DB8910A32FB03BD640103F6
2,2017-12-01T06:13:14Z,"[{'player': 'me', 'turn': 1, 'card': {'id': 'C...",False,585.0,Priest,None,97910404,None,casual,None,Mage,Secret,NaN,Americas,loss,66FFB43E8DB8910A32FB03BD640103F6
3,2017-12-01T02:23:44Z,"[{'player': 'opponent', 'turn': 1, 'card': {'i...",True,456.0,Priest,None,97901928,None,casual,None,Warrior,Taunt,NaN,Americas,loss,66FFB43E8DB8910A32FB03BD640103F6
4,2017-12-01T02:15:59Z,"[{'player': 'opponent', 'turn': 1, 'card': {'i...",False,729.0,Priest,None,97901622,None,casual,None,Druid,None,NaN,Americas,loss,66FFB43E8DB8910A32FB03BD640103F6


In [4]:
opponentCards = []
meCards = []
for game in rawDf['card_history']:
    meCardsPlayed = {}
    opponentCardsPlayed = {}
    dictSelect = {'me':meCardsPlayed, 'opponent':opponentCardsPlayed}
    for action in game:
        card = action['card']['name']
        dictionary = dictSelect[action['player']]
        dictionary[str(card)] = 1#dictionary.get(card, 0) + 1
    
    meCards.append(meCardsPlayed)
    opponentCards.append(opponentCardsPlayed)

In [5]:
meDf = pd.DataFrame(meCards)#.add_suffix('_me')
opponentDf = pd.DataFrame(opponentCards)#.add_suffix('_opponent')
df = meDf.append(opponentDf)
df = df.fillna(0).astype('int')
df.head()

,"""Little Friend""",A Light in the Darkness,Aberrant Berserker,Abominable Bowman,Abomination,Abusive Sergeant,Abyssal Enforcer,Acherus Veteran,Acidic Swamp Ooze,Acidmaw,...,Young Dragonhawk,Young Priestess,Youthful Brewmaster,Ysera,Ysera Awakens,Zarog's Crown,Zealous Initiate,Zola the Gorgon,Zombie Chow,Zoobot
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
# remove hero powers
removeCards = ['Shapeshift', 'Steady Shot', 'Fireblast', 
               'Reinforce', 'Lesser Heal', 'Dagger Mastery', 
               'Totemic Call', 'Life Tap', 'Armor Up!', 'The Coin']
df = df.drop(removeCards, axis=1)

In [7]:
frequentSets = apriori(df, min_support=0.02, use_colnames=True)

In [28]:
rules = association_rules(frequentSets, metric='confidence', min_threshold=0.4)
rules.shape

(1795, 9)

In [31]:
rules.sort_values('confidence', ascending=False)

,antecedants,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1634,"(Flame Elemental, Southsea Captain, Patches th...",(Fire Fly),0.028534,0.097561,0.028534,1.000000,10.250000,0.025750,inf
1075,"(Flame Elemental, Southsea Captain)",(Fire Fly),0.030310,0.097561,0.030271,0.998698,10.236654,0.027314,693.073171
1077,"(Southsea Deckhand, Flame Elemental)",(Fire Fly),0.020680,0.097561,0.020601,0.996183,10.210878,0.018584,236.439024
1071,"(Flame Elemental, Patches the Pirate)",(Fire Fly),0.054622,0.097561,0.054385,0.995665,10.205564,0.049056,208.162602
304,(Kazakus Potion),(Kazakus),0.035165,0.042071,0.034967,0.994388,23.635846,0.033488,170.702913
1324,"(Swashburglar, Southsea Captain)",(Patches the Pirate),0.021233,0.197174,0.020404,0.960967,4.873693,0.016218,20.567632
1078,"(Fire Fly, Southsea Captain)",(Patches the Pirate),0.036072,0.197174,0.034099,0.945295,4.794214,0.026986,14.675656
1631,"(Fire Fly, Flame Elemental, Southsea Captain)",(Patches the Pirate),0.030271,0.197174,0.028534,0.942634,4.780715,0.022566,13.994713
1109,"(Flame Elemental, Southsea Captain)",(Patches the Pirate),0.030310,0.197174,0.028534,0.941406,4.774490,0.022558,13.701560
1637,"(Flame Elemental, Southsea Captain)","(Fire Fly, Patches the Pirate)",0.030310,0.064212,0.028534,0.941406,14.660941,0.026588,15.970784


In [23]:
indices = ['' in itemSet for itemSet in rules['antecedants']]
indices = pd.Series(indices)
print(indices.sum(), 'results')
rules[indices][['antecedants', 'consequents', 'confidence']].sort_values('confidence', ascending=False)

0 results


,antecedants,consequents,confidence
